In [ ]:
from random import randint

from lightfm import LightFM
import pandas as pd
import numpy as np
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import LightFMWrapperModel

from service.models.ann import ApproximateNearestNeighbors

# LOAD DATA

In [ ]:
!mkdir ../data
!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip -O ../data/data_original.zip
!unzip ../data/data_original.zip -d ../data

In [ ]:
interactions = pd.read_csv('../data/kion_train/interactions.csv')
users = pd.read_csv('../data/kion_train/users.csv')
items = pd.read_csv('../data/kion_train/items.csv')

# Preprocess

In [ ]:
Columns.Datetime = 'last_watch_dt'

In [ ]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [ ]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [ ]:
max_date = interactions[Columns.Datetime].max()

In [ ]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [ ]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

In [ ]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [ ]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [ ]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Models

In [ ]:
dataset = Dataset.construct(interactions_df=train)

In [ ]:
TEST_USERS = test[Columns.User].unique()

# Добавляем аватаров в обучающую выборку

1. Фанат фильмов с Киану Ривзом, смотрит фильмы и рекламу, только с его участием!
2. Человек-Патриот, смотрит фильмы только made in Russia. Считает что раньше было лучше!
3. Девочка, смотрит только мультфильмы про принцесс и ждёт своего принца на белом коне.

In [ ]:
kianu_fanboy = items[(items.actors.isna() == False) & (items.actors.str.contains('Киану Ривз'))][:10]
patriot = items[(items.countries.isna() == False) & (items.countries.str.contains('Россия|СССР'))][:10]
princesses = items[(items.age_rating <= 12) & (items.keywords.str.contains('принцесса|королева', case=False))][:30]

new_items = [kianu_fanboy, patriot, princesses]

In [ ]:
train_2 = train.copy()
max_user_id = train_2.user_id.max()
avatar_ids = list()

for user_items in new_items:
    max_user_id += 1
    for item_id in user_items.item_id:
        last_watch_dt = f'2022-{str(randint(1,12)).zfill(2)}-{str(randint(1,28)).zfill(2)}'
        if max_user_id not in avatar_ids:
            avatar_ids.append(max_user_id)
        train_2 = train_2.append({
            'user_id': max_user_id,
            'item_id': item_id,
            'last_watch_dt': last_watch_dt,
            'watched_pct': randint(70, 100),
            'weight': 3
        }, ignore_index=True)

In [ ]:
avatar_map = pd.DataFrame({"user_id": avatar_ids, "name": ['kianu', 'patriot', 'pricesses']})

In [ ]:
avatar_ids

# Обучение LightFm

In [ ]:
dataset_2 = Dataset.construct(interactions_df=train_2)

In [ ]:
 model = LightFMWrapperModel(
        LightFM(
            no_components=12,
            loss='warp',
            random_state=1234,
        ),
        epochs=10,
        num_threads=4,
    )

model.fit(dataset_2)

In [ ]:
recoms = model.recommend(
    users=avatar_ids,
    dataset=dataset_2,
    k=10,
    filter_viewed=True,
)
recoms = pd.merge(recoms, items, on='item_id')
recoms = pd.merge(recoms, avatar_map, on='user_id')[['name', 'countries', 'title', 'genres', 'age_rating', 'actors', 'keywords']]


# Approximate Nearest Neighbors

In [ ]:
ann = ApproximateNearestNeighbors(model=model, dataset=dataset_2)
ann.fit(k_reco=10)

## Фанат Киану Ривза
Модель плохо справилась, в плане предсказания фильмов с данным актёром, однако в целом попадает в в жанры, в которых он снимался

In [ ]:
lightfm_recs_kianu = recoms[:10]
ann_recs_kianu = items[items.item_id.isin(ann.predict(user_id=avatar_idas[0]))]

In [ ]:
assert set(lightfm_recs_kianu.title) == set(ann_recs_kianu.title)

In [ ]:
lightfm_recs_kianu

## Человек-патриот
Все фильмы, кроме одного из России, следовательно модель справилась достаточно хорошо

In [ ]:
lightfm_recs_patr = recoms[10:20]
ann_recs_patr = items[items.item_id.isin(ann.predict(user_id=avatar_ids[1]))]

In [ ]:
assert set(lightfm_recs_patr.title) == set(ann_recs_patr.title)

In [ ]:
lightfm_recs_patr

## Ребенок, смотрящий мультики про принцесс
В рекомендациях только мультики для детей, так что в "ребёнка" попали, однако из всего списка только 1 мультфильм про принцесс.

In [ ]:
lightfm_recs_child = recoms[20:30]
ann_recs_child = items[items.item_id.isin(ann.predict(user_id=avatar_ids[2]))]

In [ ]:
assert set(lightfm_recs_child.title) == set(ann_recs_child.title)

In [ ]:
ann_recs_child

ВЫВОД: Получили быстрее работающие предикты, без потери качества предсказаний

In [19]:
avatar_map = pd.DataFrame({"user_id": avatar_ids, "name": ['kianu', 'patriot', 'pricesses']})

In [20]:
avatar_ids

[1097558, 1097559, 1097560]

# Обучение LightFm

In [21]:
dataset_2 = Dataset.construct(interactions_df=train_2)

/usr/local/lib/python3.8/dist-packages/rectools/dataset/interactions.py:62: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df[Columns.Datetime] = df[Columns.Datetime].astype("datetime64[ns]")


In [22]:
 model = LightFMWrapperModel(
        LightFM(
            no_components=12,
            loss='warp',
            random_state=1234,
        ),
        epochs=10,
        num_threads=4,
    )

model.fit(dataset_2)

In [23]:
recoms = model.recommend(
    users=avatar_ids,
    dataset=dataset_2,
    k=10,
    filter_viewed=True,
)
recoms = pd.merge(recoms, items, on='item_id')
recoms = pd.merge(recoms, avatar_map, on='user_id')[['name', 'countries', 'title', 'genres', 'age_rating', 'actors', 'keywords']]


# Approximate Nearest Neighbors

In [24]:
ann = ApproximateNearestNeighbors(model=model, dataset=dataset_2)
ann.fit(k_reco=10)

## Фанат Киану Ривза
Модель плохо справилась, в плане предсказания фильмов с данным актёром, однако в целом попадает в в жанры, в которых он снимался

In [32]:
lightfm_recs_kianu = recoms[:10]
ann_recs_kianu = items[items.item_id.isin(ann.predict(user_id=avatar_idas[0]))]

In [38]:
assert set(lightfm_recs_kianu.title) == set(ann_recs_kianu.title)

In [45]:
lightfm_recs_kianu

,name,countries,title,genres,age_rating,actors,keywords
0,kianu,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
1,kianu,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
2,kianu,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
3,kianu,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
4,kianu,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
5,kianu,Россия,Секреты семейной жизни,комедии,18.0,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...","брызги крови, кровь, жестокое обращение с живо..."
6,kianu,США,Веном,"популярное, фантастика, триллеры, боевики, ужасы",16.0,"Том Харди, Вуди Харрельсон, Уэйд Уильямс, Мише...","Сан-Франциско, Калифорния, космический корабль..."
7,kianu,"США, Франция",Ford против Ferrari,драмы,16.0,"Кристиан Бэйл, Мэтт Дэймон, Катрина Балф, Трэй...","по мотивам романа или книги, биография, спорт,..."
8,kianu,Россия,День города,комедии,16.0,"Катерина Шпица, Антон Филипенко, Павел Ворожцо...","2021, россия, день, города"
9,kianu,"Великобритания, США",Kingsman: Секретная служба,"боевики, криминал, приключения, комедии",18.0,"Тэрон Эджертон, Колин Фёрт, Сэмюэл Л. Джексон,...","шпион, великобритания, секретная организация, ..."


## Человек-патриот
Все фильмы, кроме одного из России, следовательно модель справилась достаточно хорошо

In [43]:
lightfm_recs_patr = recoms[10:20]
ann_recs_patr = items[items.item_id.isin(ann.predict(user_id=avatar_ids[1]))]

In [44]:
assert set(lightfm_recs_patr.title) == set(ann_recs_patr.title)

In [46]:
lightfm_recs_patr

,name,countries,title,genres,age_rating,actors,keywords
10,patriot,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
11,patriot,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
12,patriot,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
13,patriot,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
14,patriot,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
15,patriot,США,Пираты карибского моря: Проклятие чёрной жемчу...,"боевики, фэнтези, приключения",12.0,"Джонни Депп, Джеффри Раш, Орландо Блум, Кира Н...","экзотический остров, кузнец, торговая компания..."
16,patriot,США,Пираты Карибского Моря: Сундук Мертвеца,"боевики, фэнтези, приключения",12.0,"Джонни Депп, Орландо Блум, Кира Найтли, Джек Д...","ведьма, гадалка, неволя, экзотический остров, ..."
17,patriot,США,Пираты карибского моря: На краю света,"боевики, фэнтези, приключения",12.0,"Джонни Депп, Джеффри Раш, Орландо Блум, Кира Н...","экзотический остров, торговая компания Восточн..."
18,patriot,Россия,Танки,"историческое, военные, приключения",12.0,"Андрей Мерзликин, Аглая Тарасова, Сергей Стука...","Вторая мировая война, Иосиф Сталин, танк, 2018..."
19,patriot,"США, Великобритания",Пираты карибского моря: На странных берегах,"боевики, фэнтези, приключения",12.0,"Джонни Депп, Пенелопа Крус, Джеффри Раш, Иэн М...","море, капитан, мятеж, меч, премьер-министр, пл..."


## Ребенок, смотрящий мультики про принцесс
В рекомендациях только мультики для детей, так что в "ребёнка" попали, однако из всего списка только 1 мультфильм про принцесс.

In [48]:
lightfm_recs_child = recoms[20:30]
ann_recs_child = items[items.item_id.isin(ann.predict(user_id=avatar_ids[2]))]

In [49]:
assert set(lightfm_recs_child.title) == set(ann_recs_child.title)

In [50]:
ann_recs_child

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
1274,5411,film,Монстры на каникулах 3: Море зовёт,Hotel Transylvania 3: Summer Vacation,2018.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,Генндий Тартаковский,"Энди Сэмберг, Селена Гомес, Кевин Джеймс, Стив...",На этот раз Дракула вместе со своими друзьями ...,"третья часть, круизное судно, персонаж Дракула..."
1554,16270,film,Тайна Коко,Coco,2017.0,"мультфильм, фэнтези, приключения",США,NaN,12.0,NaN,"Ли Анкрич, Эдриан Молина","Энтони Гонсалес, Гаэль Гарсиа Берналь, Бенджам...",Мексиканский юноша Мигель живёт в семье сапожн...,"Мексика, гитара, музыкант, скелет, музыка, заг..."
9085,1105,film,Рапунцель: Запутанная история,Tangled,2010.0,"мультфильм, приключения, мюзиклы, мелодрамы, ф...",США,NaN,6.0,NaN,"Нэйтан Грено, Байрон Ховард","Мэнди Мур, Закари Ливай, Донна Мерфи, Рон Перл...",Обаятельный разбойник Флинн путешествует по жи...,"Заложник, магия, лошадь, сказка, мюзикл, блонд..."
9698,16166,film,Зверополис,Zootopia,2016.0,"приключения, мультфильм, детективы, комедии",США,NaN,6.0,NaN,"Байрон Ховард, Рич Мур, Джаред Буш","Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...",Добро пожаловать в Зверополис — современный го...,"аллегория, лев, бегемот, лиса, слон, овца, бел..."
9921,10761,film,Моана,Moana,2016.0,"мультфильм, фэнтези, мюзиклы",США,NaN,6.0,NaN,"Рон Клементс, Джон Маскер, Дон Холл","Аулии Кравальо, Дуэйн Джонсон, Рэйчел Хаус, Те...","Бесстрашная Моана, дочь вождя маленького племе...","океан, парусник, море, мифология, остров, мюзи..."
11653,10323,film,Университет монстров,Universitet monstrov,2013.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,Дэн Скэнлон,"Джон Гудман, Билли Кристал, Стив Бушеми, Хелен...",Майк и Салли — самые опытные пугатели в Монстр...,"соревнование, монстр, тренировочный лагерь, ко..."
12034,2956,film,Король Лев,The Lion King,1994.0,"драмы, мультфильм, приключения, мюзиклы",США,NaN,0.0,NaN,"Роджер Аллерс, Роб Минкофф","Мэттью Бродерик, Джереми Айронс, Нэйтан Лейн, ...",У величественного Короля-Льва Муфасы рождается...,"потеря любимого, лев, мюзикл, кабан, дядя, шам..."
12035,15266,film,Корпорация монстров,Monsters University,2001.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,"Пит Доктер, Дэвид Силверман, Ли Анкрич","Джон Гудман, Билли Кристал, Мэри Гиббс, Стив Б...","Склизкий гад в сливном бачке, мохнатый зверь, ...","чудовище, обман, похищение, младенец, злодей, ..."
12620,13243,film,Головоломка,Inside Out,2015.0,"фантастика, мультфильм, комедии",США,NaN,6.0,NaN,"Пит Доктер, Роналдо Дель Кармен","Эми Полер, Филлис Смит, Ричард Кайнд, Билл Хей...",11-летняя девочка Райли переезжает с семьёй из...,"мечта, мультфильм, воображаемый друг, начальна..."
15352,7582,film,Холодное сердце II,Frozen II,2019.0,"фэнтези, мультфильм, музыкальные",США,NaN,6.0,NaN,"Крис Бак, Дженнифер Ли","Идина Мензел, Кристен Белл, Джонатан Грофф, Дж...","Анна, Эльза, Кристоф, его верный олень Свен и ...","королева, магия, королевство, плотина, дух, же..."


ВЫВОД: Получили быстрее работающие предикты, без потери качества предсказаний